# Bot Functions

This notebook is dedicated to the setup of Bot1 (similar to Project 2). The functions defined in this notebook can be imported to the Model1, Model2, and Model3 notebooks to generate datasets or run simulations specific to each use case.

## Preliminary Setup

### Imports

In [1]:
import os
import numpy as np
import random
import math
import matplotlib.pyplot as plt

### Grid Setup

In [2]:
# Function to find current open cells in grid
def find_all_open_cells(grid):
    open_cells = set()

    for i in range(30):
        for j in range(30):
            if grid[i, j] == 0:
                open_cells.add((i, j))

    return open_cells

In [3]:
# Function to check if a cell has one open neighbor
def single_open_neighbor_check(i, j, grid):
    count = 0

    if (i + 1 < 30) and (grid[i + 1, j] == 0):
        count += 1
    if (i - 1 >= 0) and (grid[i - 1, j] == 0):
        count += 1
    if (j + 1 < 30) and (grid[i, j + 1] == 0):
        count += 1
    if (j - 1 >= 0) and (grid[i, j - 1] == 0):
        count += 1

    return count == 1

In [4]:
# Function to open a random blocked cell with one open neighbor
def open_random_single_neighbor_cell(grid, num_open_neighbor_cells):
    open_neighbor_cells = set()

    # Find open neighbor cells
    for i in range(30):
        for j in range(30):
            if (grid[i, j] == 1) and (single_open_neighbor_check(i, j, grid)):
                open_neighbor_cells.add((i, j))
    
    # Store total number of open neighbor cells
    num_open_neighbor_cells = len(open_neighbor_cells)

    # No more single open neighbor cells
    if num_open_neighbor_cells == 0:
        return grid, num_open_neighbor_cells
    
    # Pick a random blocked cell from set and open it
    random_cell = random.choice(tuple(open_neighbor_cells))
    grid[random_cell] = 0

    return grid, num_open_neighbor_cells

In [5]:
# Function to open a random closed neighbor
def open_random_closed_neighbor(i, j, grid):
    closed_neighbors = set()

    if (i + 1 < 30) and (grid[i + 1, j] == 1):
        closed_neighbors.add((i + 1, j))
    if (i - 1 >= 0) and (grid[i - 1, j] == 1):
        closed_neighbors.add((i - 1, j))
    if (j + 1 < 30) and (grid[i, j + 1] == 1):
        closed_neighbors.add((i, j + 1))
    if (j - 1 >= 0) and (grid[i, j - 1] == 1):
        closed_neighbors.add((i, j - 1))

    if len(closed_neighbors) > 0:
        random_neighbor = random.choice(tuple(closed_neighbors))
        grid[random_neighbor[0], random_neighbor[1]] = 0

    return grid

In [6]:
# Function to create an empty 30x30 grid
def create_grid():
    grid = np.full((30, 30), 1) # Create a new 30x30 numpy matrix (2D array)
    grid[random.randrange(1, 29), random.randrange(1, 29)] = 0 # Open a random blocked cell (except on edges and corners)

    num_open_neighbor_cells = 900

    # Iteratively open all single open neighbor cells
    while (num_open_neighbor_cells > 0):
        grid, num_open_neighbor_cells = open_random_single_neighbor_cell(grid, num_open_neighbor_cells)
    
    dead_end_cells = set()

    # Find dead-end cells
    for i in range(30):
        for j in range(30):
            if (grid[i, j] == 0) and (single_open_neighbor_check(i, j, grid)):
                dead_end_cells.add((i, j))

    # Keep roughly half of the dead-end cells found
    dead_end_cells = set(random.choices(list(dead_end_cells), k=(len(dead_end_cells) // 2)))

    # Open a random closed neighbor for each of the dead-end cells in the set
    for i in dead_end_cells:
        grid = open_random_closed_neighbor(i[0], i[1], grid)

    # Find current open cells in grid
    open_cells = find_all_open_cells(grid)

    return grid, open_cells

grid, open_cells = create_grid()

# print(f"Grid:\n{grid}\nOpen Cells:\n{open_cells}")

In [7]:
# Open up all unblocked cells in grid
def reset_grid(grid, open_cells):
    for i in range(30):
        for j in range(30):
            if grid[i, j] != 1:
                grid[i, j] = 0
    
    open_cells = find_all_open_cells(grid)
    return grid, open_cells

### Bot, Alien, and Crew Placement

In [8]:
# Function to place the bot at a random open cell in the grid
def place_bot(grid, open_cells):
    bot = random.choice(tuple(open_cells)) # Pick a random cell
    grid[bot[0], bot[1]] = 2 # Place bot in grid and label space as 2
    open_cells.remove(bot)

    return bot, grid, open_cells

In [9]:
# Function to place the crew at a random open cell in the grid (must occur after placing bot)
def place_crew(grid, open_cells, crew_list):
    crew = random.choice(tuple(open_cells - set(crew_list))) # Pick a random cell without an existing crew member
    grid[crew[0], crew[1]] = 4 # Place crew member in grid and label space as 4
    crew_list.append(crew)

    return crew_list, grid

In [10]:
# Function to place an alien at a unoccupied cell in the grid
def place_alien(grid, open_cells, alien_list, bot, detect_square_k):
    alien = None
    # Make sure alien is not in detection square
    while True:
        alien = random.choice(tuple(open_cells - set(alien_list))) # Pick open cell that is unoccupied by another alien
        if not ((alien[0] >= (bot[0]-detect_square_k)) and (alien[0] <= (bot[0]+detect_square_k))) and not ((alien[1] >= (bot[1]-detect_square_k)) and (alien[1] <= (bot[1]+detect_square_k))):
            break
    
    grid[alien[0]][alien[1]] = 3 # Place alien in grid and denote space as 3
    alien_list.append(alien)
    return alien_list, grid

### Alien Movement

In [11]:
# Function to ensure that potential cells to move to is in bounds
def check_valid_neighbors(dim, x_coord, y_coord):
    neigh_list = []
    neigh_list.append((x_coord+1, y_coord))
    neigh_list.append((x_coord-1, y_coord))
    neigh_list.append((x_coord, y_coord+1))
    neigh_list.append((x_coord, y_coord-1))
    main_list = []
    for neigh in neigh_list:
        if neigh[0] <= dim-1 and neigh[0] >= 0 and neigh[1] <= dim-1 and neigh[1] >= 0:
            main_list.append(neigh)
    return main_list

In [12]:
# Move aliens randomly to adjacent cells
def move_aliens(grid, alien_list, bot):
    # random.shuffle(alien_list)
    new_position = []
    marker = 0
    for alien in alien_list:
        possible_moves = check_valid_neighbors(len(grid), alien[0], alien[1])
        possible_moves.append(alien)
        # Get all spatially possible coordinates that the alien can move to 
        while possible_moves:
            current = random.choice(possible_moves)
            # Check if cell is open 
            if grid[current[0]][current[1]] == 1:
                possible_moves.remove(current)
                continue
            # Check if alien captures bot
            if current == bot:
                marker = 1
                new_position.append(current)
                new_position.extend(alien_list[alien_list.index(alien)+1: len(alien_list)])
                return marker, new_position, grid
            
            grid[alien[0]][alien[1]] = 0
            grid[current[0]][current[1]] = 3
            new_position.append(current)
            break 
        
    return marker, new_position, grid

### Sensor Setup

In [13]:
# Sensor to detect aliens within a (2k + 1) x (2k + 1) square around bot
def alien_sensor(alien_list, bot, k):
    bot_x_max = min(bot[0] + k, 29) # k cells to the right of bot
    bot_x_min = max(0, bot[0] - k) # k cells to the left of bot
    bot_y_max = min(bot[1] + k, 29) # k cells to the top of bot
    bot_y_min = max(0, bot[1] - k) # k cells to the bottom of bot

    # Check if each alien is within the detection square
    for alien in alien_list:
        if (alien[0] >= bot_x_min and alien[0] <= bot_x_max) and (alien[1] >= bot_y_min and alien[1] <= bot_y_max):
            return True

    return False

In [14]:
# Updated Crew Sensor to avoid having to run BFS every step
def crew_sensor(grid, bot, alpha, d_lookup_table, crew_list):
    d_dict = {}

    # Case in which bot has never been to cell before
    if d_lookup_table.get(bot) == None:
        seen_cells = set()
        bfs_queue = []
        bfs_queue.append(bot)
        seen_cells.add(bot)
        d_dict[bot[0], bot[1]] = 0

        # Use BFS to find shortest path cost from bot to all cells
        while len(bfs_queue) > 0:
            curr_cell = bfs_queue.pop(0)

            neighbors = check_valid_neighbors(30, curr_cell[0], curr_cell[1])

            for neighbor in neighbors:
                if grid[neighbor[0], neighbor[1]] != 1 and neighbor not in seen_cells:
                    seen_cells.add(neighbor)
                    bfs_queue.append(neighbor)
                    d_dict[neighbor[0], neighbor[1]] = d_dict[curr_cell[0], curr_cell[1]] + 1 # Set distance of neighbor to current cell's distance + 1
        d_lookup_table[bot] = d_dict # Forgot this line I think - Aditya (Thanks for adding! - Rishik)

    # Case in which bot has been to cell before (and knows distance to closest crew member)
    else:
        d_dict = d_lookup_table[bot]

    d_min = 901

    # Find d for closest crew member
    for crew_member in crew_list:
        if d_dict[crew_member] < d_min:
            d_min = d_dict[crew_member]
    
    if d_min == 901:
        return False, d_lookup_table # Don't beep if no crew member found

    prob = math.exp(-alpha * (d_min - 1))
    
    return np.random.choice([True, False], p=[prob, 1 - prob]), d_lookup_table # Beep with the specified probability

### Probability Matrix Setup

In [15]:
# Create alien probability matrix (dictionary) for t = 0
def initialize_alienmatrix(open_cells, bot, k):
    open_cells.add(bot)

    bot_x_max = min(bot[0] + k, 29) # k cells to the right of bot
    bot_x_min = max(0, bot[0] - k) # k cells to the left of bot
    bot_y_max = min(bot[1] + k, 29) # k cells to the top of bot
    bot_y_min = max(0, bot[1] - k) # k cells to the bottom of bot

    alien_matrix = {}
    for cell in open_cells:
        if bot_x_min <= cell[0] <= bot_x_max and bot_y_min <= cell[1] <= bot_y_max:
            alien_matrix[cell] = 0
        else:
            alien_matrix[cell] = 1  # Temporary value, to be normalized later

    # Normalize probabilities for cells where the alien can be
    valid_cells_count = len(open_cells) - sum(value == 0 for value in alien_matrix.values())
    for cell, prob in alien_matrix.items():
        if prob != 0:
            alien_matrix[cell] = 1 / valid_cells_count

    return alien_matrix

In [16]:
# Create crew probability matrix (dictionary) for t = 0
def initialize_crewmatrix(open_cells, crew_list, bot):
    open_cells.add(bot)
    # Crew member can be at any open cell except the ones occupied by the bot or another crew
    inital_prob = [1/(len(open_cells) - (1 + (len(crew_list)-1)))] * len(open_cells)
    crew_matrix = dict(zip(open_cells, inital_prob))
    bot_cell = {bot : 0}
    crew_matrix.update(bot_cell)
    open_cells.remove(bot)

    return crew_matrix

### Probability Matrix Updates

In [17]:
# Update probabilties for alien matrix based on beep
def update_alienmatrix(alien_matrix, detected, bot, k):
    alien_keys = set(alien_matrix.keys())

    if detected:
        # Cells outside detection square should have probability 0
        out_detection_cells = {key for key in alien_keys if not ((bot[0]-k <= key[0] <= bot[0]+k) and (bot[1]-k <= key[1] <= bot[1]+k))}
        for cell in out_detection_cells:
            alien_matrix[cell] = 0

        # Normalize probabilities inside detection square
        in_detection_cells = alien_keys - out_detection_cells
        prob_sum = sum(alien_matrix[cell] for cell in in_detection_cells)
        if prob_sum != 0:
            for cell in in_detection_cells:
                alien_matrix[cell] /= prob_sum
    else:
        # Cells inside detection square should have probability 0
        detection_cells = {key for key in alien_keys if (bot[0]-k <= key[0] <= bot[0]+k) and (bot[1]-k <= key[1] <= bot[1]+k)}
        for cell in detection_cells:
            alien_matrix[cell] = 0

        # Normalize probabilities outside detection square
        outside_cells = alien_keys - detection_cells
        prob_sum = sum(alien_matrix[cell] for cell in outside_cells)
        if prob_sum != 0:
            for cell in outside_cells:
                alien_matrix[cell] /= prob_sum

    return alien_matrix

In [18]:
# Update probabilties for crew matrix based on beep
def update_crewmatrix(crew_matrix, detected, d_lookup_table, bot, alpha):
    # Case where beep is detected from bot cell
    if detected:
        d_dict = d_lookup_table.get(bot) # Get the d dictionary calculated with the crew sensor
        total_summation = 0
        for cell in crew_matrix:
            d = d_dict.get(cell) # Find d from bot to cell
            if cell == bot:
                crew_matrix[cell] = 0 # Crew member not at current cell
            else:
                crew_matrix[cell] *= math.exp(-alpha * (d - 1)) # Multiply probability of cell containing crew by given prob
            total_summation += crew_matrix[cell] # Calculate sum of all probabilities
        
        for key in crew_matrix:
            crew_matrix[key] /= total_summation # Normalize probabilities
    # Case where beep is not detected from bot cell
    else:
        d_dict = d_lookup_table.get(bot) # Get the d dictionary calculated with the crew sensor
        total_summation = 0
        for cell in crew_matrix:
            d = d_dict.get(cell) # Find d from bot to cell
            if cell == bot:
                crew_matrix[cell] = 0 # Crew member not at current cell
            else:
                crew_matrix[cell] *= (1 - math.exp(-alpha * (d - 1))) # Multiply probability of cell containing crew by 1 - given prob
            total_summation += crew_matrix[cell] # Calculate sum of all probabilities
        
        for key in crew_matrix:
            crew_matrix[key] /= total_summation # Normalize probabilities

    return crew_matrix

In [19]:
# Recalculate probability matrices after bot move    
def update_afterbotmove(bot, alien_matrix, crew_matrix):
    # Prior Probability alien not in current cell
    alienprob_not = 1 - alien_matrix[bot]
    # Prob alien not in current cell
    crew_probnot = 1 - crew_matrix[bot]
    # Bot did not capture or die so we know current cell does not have crew or alien
    alien_matrix[bot] = 0
    crew_matrix[bot] = 0
    
    for cell in alien_matrix:
        alien_matrix[cell] = alien_matrix[cell] / alienprob_not
    for cell in crew_matrix:
        crew_matrix[cell] = crew_matrix[cell] / crew_probnot

    return alien_matrix, crew_matrix

In [20]:
# Update probabilities after alien moves
def update_afteralienmove(ship, alien_list, alien_matrix):
    total_summation = 0
    neighbors = check_valid_neighbors(len(ship), alien_list[0][0], alien_list[0][1])
    neighbors = [neigh for neigh in neighbors if ship[neigh[0], neigh[1]] != 1]
    for neigh in neighbors:
        if neigh in alien_matrix:
            total_summation = total_summation + (alien_matrix[neigh] * (1/(len(neighbors)+1))) # Plus 1 because alien could stay in place 
    
    alien_matrix[alien_list[0]] = total_summation
    return alien_matrix

### Bot Movement

In [21]:
# Function to move bot to specified cell (Makes sense to do probability updates and move decision in functions for each Bot, since other factors to consider)
def move_bot(grid, bot, new_cell, crew_list, alien_list, open_cells, win_count, bot_num):
    # Add new bot location to open cells set and remove old one. Modify grid accordingly
    open_cells.add(bot)
    grid[bot[0], bot[1]] = 0
    grid[new_cell[0], new_cell[1]] = 2
    open_cells.remove(new_cell)
    bot = new_cell
    marker = 0

    # Case where bot lands on the same cell as a crew member
    for crew_member in crew_list:
        if bot == crew_member:
            # ^ In the case of 2 crew members, this only matters if bot has saved both crew members. Shall evaluate probability of saving crew as mentioned in https://cs520f23.zulipchat.com/#narrow/stream/409248-project-2/topic/Questions.20on.20the.20evaluation/near/401403344
            crew_list.remove(crew_member)

            if bot_num <= 2:
                win_count += 1 # Increment win count because crew member has been saved
                crew_list, grid = place_crew(grid, open_cells, crew_list)
            else:
                if len(crew_list) == 0:
                    win_count += 1 # Increment win count because both crew members have been saved
                    crew_list, grid = place_crew(grid, open_cells, crew_list)
                    crew_list, grid = place_crew(grid, open_cells, crew_list)

            return bot, crew_list, grid, open_cells, win_count, marker
        
    # Case where bot lands on the same cell as an alien
    for alien in alien_list:
        if bot == alien:
            marker = 1

            return bot, crew_list, grid, open_cells, win_count, marker
    
    return bot, crew_list, grid, open_cells, win_count, marker

In [22]:
# Determine the best neighboring cell for the bot to move to based on probability matrices
def determine_move(moves, alien_matrix, crew_matrix):
    zero_alienprob = [move for move in moves if alien_matrix[move] == 0]
    nonzero_crewprob = [move for move in moves if crew_matrix[move] != 0]

    def find_max_prob_cell(cell_list, matrix):
        max_prob = -1
        candidates = []
        for cell in cell_list:
            current_prob = matrix[cell]

            # Find max probability cell
            if current_prob > max_prob:
                max_prob = current_prob
                candidates = [cell] # Reset list of highest probability cells if new max found
            elif current_prob == max_prob:
                candidates.append(cell) # Add cell to list if same probability as max

        return random.choice(candidates) if candidates else None

    # Case at least one move with 0 alien probability
    if zero_alienprob:
        chosen_move = find_max_prob_cell(zero_alienprob, crew_matrix)
    # Case where at least one move with nonzero crew probability
    elif nonzero_crewprob:
        chosen_move = find_max_prob_cell(nonzero_crewprob, crew_matrix)
    else:
        chosen_move = random.choice(moves)

    return chosen_move